In [1]:
import requests
from bs4 import BeautifulSoup
import sqlite3
import pandas as pd

In [2]:
# SUUMOのURL
url = "https://suumo.jp/jj/chintai/ichiran/FR301FC011/?ar=030&bs=040&fw=%E5%8D%83%E4%BB%A3%E7%94%B0%E5%8C%BA"

In [3]:
# リクエストを送信
response = requests.get(url)

In [4]:
# BeautifulSoupオブジェクトを作成
soup = BeautifulSoup(response.text, 'html.parser')

In [5]:
# BeautifulSoupオブジェクトから整形されたHTMLを取得
pretty_html = soup.prettify()

# 出力
print(pretty_html)

<!DOCTYPE html>
<!--[if lt IE 7]><html lang="ja" class="ie"><![endif]-->
<!--[if IE 7]><html lang="ja" class="ie"><![endif]-->
<!--[if IE 8]><html lang="ja" class="ie"><![endif]-->
<!--[if gt IE 8]><!-->
<html lang="ja">
 <!--<![endif]-->
 <!-- FR301FC0110 START -->
 <head>
  <meta charset="utf-8"/>
  <!-- JJ901FI0110 -->
  <title>
   千代田区に一致する賃貸・部屋探し情報一覧｜賃貸マンション・賃貸アパートはSUUMO(スーモ)賃貸
  </title>
  <meta content="千代田区,賃貸,賃貸情報,賃貸マンション,賃貸アパート,賃貸物件,賃貸住宅,部屋探し,SUUMO(スーモ)" name="keywords"/>
  <meta content="リクルートの不動産・住宅サイトSUUMO(スーモ)。千代田区の賃貸一覧。千代田区の賃貸を探すならSUUMO(スーモ)賃貸" name="description"/>
  <link href="/front/img/favicon.ico" rel="shortcut icon" type="image/vnd.microsoft.icon"/>
  <link href="/front/img/favicon.ico" rel="icon" type="image/vnd.microsoft.icon"/>
  <meta content="noarchive" name="robots"/>
  <!-- JJ901FI0110 -->
  <link href="/edit/assets/suumo/css/fr-list.css?1699941564000" media="screen" rel="stylesheet" type="text/css"/>
  <!--[if lt IE 9]>
<script src="/edit/assets/allsites/js

In [6]:
# 物件情報を辞書形式で準備
property_data = {
    '物件名': [soup.find('a', class_='js-cassetLinkHref').text],
    'URL': ['https://suumo.jp' + soup.find('a', class_='js-cassetLinkHref')['href']],
    'イメージ画像': [soup.find('img', class_='js-noContextMenu js-linkImage js-scrollLazy js-adjustImg')['rel']],
    '賃料': [int(float(soup.find('div', class_='detailbox-property-point').text.replace('万円', '')) * 10000)],
    '管理費': [soup.find('td', class_='detailbox-property--col1').find_all('div')[1].text.replace('管理費 ', '')],
    '間取り': [soup.find('td', class_='detailbox-property--col3').find_all('div')[0].text],
    '専有面積': [soup.find('td', class_='detailbox-property--col3').find_all('div')[1].text.replace('\n', '').strip()],
    '向き': [soup.select('td.detailbox-property--col3')[0].find_all('div')[2].text if len(soup.select('td.detailbox-property--col3')[0].find_all('div')) > 2 else '-'],
    '物件種別': [soup.select('td.detailbox-property--col3')[1].find_all('div')[0].text],
    '築年数': [soup.select('td.detailbox-property--col3')[1].find_all('div')[1].text if len(soup.select('td.detailbox-property--col3')[1].find_all('div')) > 1 else '-'],
    '住所': [soup.select_one('td.detailbox-property-col:not([class*="detailbox-property--"])').text.strip()],
    '最寄り駅と距離': [soup.select_one('.detailnote-box > div').text.strip()],
    '敷金': ['-' if soup.find('div', string='敷') is None or soup.find('div', string='敷').find_next('div').text == '-' else soup.find('div', string='敷').find_next('div').text],
    '礼金': ['-' if soup.find('div', string='礼') is None or soup.find('div', string='礼').find_next('div').text == '-' else soup.find('div', string='礼').find_next('div').text],
    '保証金': ['-' if soup.find('div', class_='detailbox-property-inactive').text.strip() == '保証金 -' else soup.find('div', class_='detailbox-property-inactive').text.strip().replace('保証金', '').strip()],
    '敷引・償却': ['-' if soup.find('div', string='敷引・償却') is None or soup.find('div', string='敷引・償却').find_next('div').text == '-' else soup.find('div', string='敷引・償却').find_next('div').text],
    '取り扱い店舗': [soup.select_one('.detailnote-box .detailnote-box-item div:nth-of-type(1)').text.strip()] if soup.select_one('.detailnote-box .detailnote-box-item div:nth-of-type(1)') else ['-'],
    '取り扱い店舗の電話番号': [soup.find('div', class_='ui-text--bold').text.strip()] if soup.find('div', class_='ui-text--bold') else ['-']
}


In [7]:
print(property_data)

{'物件名': ['イシカワビレッジ 3階'], 'URL': ['https://suumo.jp/chintai/bc_100192305263/'], 'イメージ画像': ['https://img01.suumo.com/front/gazo/fr/bukken/263/100192305263/100192305263_gw.jpg'], '賃料': [15000], '管理費': ['7000円'], '間取り': ['ワンルーム'], '専有面積': ['20m2'], '向き': ['-'], '物件種別': ['マンション'], '築年数': ['築32年'], '住所': ['茨城県水戸市石川１'], '最寄り駅と距離': ['ＪＲ常磐線/赤塚駅 歩29分'], '敷金': ['-'], '礼金': ['-'], '保証金': ['-'], '敷引・償却': ['-'], '取り扱い店舗': ['不動産通信社 (株)ユウラク'], '取り扱い店舗の電話番号': ['03-5205-8424']}


In [8]:
# DataFrameを作成
df = pd.DataFrame(property_data)

# DataFrameを表示
print(df)

           物件名                                        URL  \
0  イシカワビレッジ 3階  https://suumo.jp/chintai/bc_100192305263/   

                                              イメージ画像     賃料    管理費    間取り  \
0  https://img01.suumo.com/front/gazo/fr/bukken/2...  15000  7000円  ワンルーム   

   専有面積 向き   物件種別   築年数         住所         最寄り駅と距離 敷金 礼金 保証金 敷引・償却  \
0  20m2  -  マンション  築32年  茨城県水戸市石川１  ＪＲ常磐線/赤塚駅 歩29分  -  -   -     -   

           取り扱い店舗   取り扱い店舗の電話番号  
0  不動産通信社 (株)ユウラク  03-5205-8424  


In [9]:
# DataFrameをCSVファイルに出力
df.to_csv('output.csv', encoding='utf-8-sig')

